In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, Activation, Dropout, Flatten, Dense
from keras.layers import GlobalMaxPooling2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K

%matplotlib inline

Using TensorFlow backend.


In [2]:
DATA_FILE = 'data/train_data.h5'
VAL_DATA_FILE = 'data/val_data.h5'
TEST_DATA_FILE = 'data/test_data.h5'
METADATA = 'metadata.npy'

metadata = np.load(METADATA).item()

In [3]:
val_f = h5py.File(VAL_DATA_FILE, 'r')

X_val = np.asarray(val_f['x'])
y_val = np.asarray(val_f['y'])

test_f = h5py.File(TEST_DATA_FILE, 'r')

X_test = np.asarray(test_f['x'])
y_test = np.asarray(test_f['y'])

In [4]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    recall = recall(y_true, y_pred)
    precision = precision(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [5]:
import pandas as pd

results = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1'])

In [6]:
train_f = h5py.File(DATA_FILE, 'r')

train_data_x = train_f['x']
train_data_y = train_f['y']

X_train = np.asarray(train_data_x)
y_train = np.asarray(train_data_y)

In [7]:
lr = 1e-3
n_conv_layers = 3
n_kernels = [128] * n_conv_layers
mp_sizes = [(2,2) for _ in range(n_conv_layers)]

model = Sequential()
model.add(BatchNormalization(axis=3, input_shape=train_data_x.shape[1:]))
for i in range(n_conv_layers):
    model.add(Conv2D(n_kernels[i], 3, 3, init='he_uniform', border_mode='same'))
    model.add(BatchNormalization(axis=3))
    model.add(Activation('relu'))
    if i < n_conv_layers - 1:
        model.add(MaxPooling2D(mp_sizes[i]))

model.add(Conv2D(8,3,3, border_mode='same'))
model.add(GlobalMaxPooling2D())
model.add(Activation('softmax'))
model.compile(Adam(lr), loss='categorical_crossentropy', metrics=['accuracy', recall, precision, f1])
model.summary()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", kernel_initializer="he_uniform")`
  if __name__ == '__main__':


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding="same")`
  from ipykernel import kernelapp as app


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 4, 7, 512)         2048      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 7, 128)         589952    
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 7, 128)         512       
_________________________________________________________________
activation_1 (Activation)    (None, 4, 7, 128)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 3, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 3, 128)         147584    
_________________________________________________________________
batch_normalization_3 (Batch (None, 2, 3, 128)         512       
__________

In [8]:
callbacks = [EarlyStopping(monitor='val_f1', min_delta=0, patience=0, verbose=0, mode='auto')]
model.fit(X_train, y_train, batch_size=64, nb_epoch=10,
          validation_data=(X_val, y_val), callbacks=callbacks)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 9133 samples, validate on 370 samples
Epoch 1/10


  64/9133 [..............................] - ETA: 206s - loss: 2.0816 - acc: 0.1094 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1: nan

 128/9133 [..............................] - ETA: 130s - loss: 2.0439 - acc: 0.1719 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1: nan

 192/9133 [..............................] - ETA: 105s - loss: 2.0357 - acc: 0.1979 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1: nan

...

8896/9133 [============================>.] - ETA: 1s - loss: 0.1541 - acc: 0.9589 - recall: 0.9448 - precision: 0.9723 - f1: 0.9582

8960/9133 [============================>.] - ETA: 0s - loss: 0.1536 - acc: 0.9590 - recall: 0.9450 - precision: 0.9725 - f1: 0.9585

9024/9133 [============================>.] - ETA: 0s - loss: 0.1539 - acc: 0.9588 - recall: 0.9448 - precision: 0.9721 - f1: 0.9581

9088/9133 [============================>.] - ETA: 0s - loss: 0.1537 - acc: 0.9588 - recall: 0.9448 - precision: 0.9722 - f1: 0.9582

9133/9133 [==============================] - 51s - loss: 0.1535 - acc: 0.9588 - recall: 0.9448 - precision: 0.9721 - f1: 0.9581 - val_loss: 0.1616 - val_acc: 0.9595 - val_recall: 0.9455 - val_precision: 0.9586 - val_f1: 0.9516

In [9]:
score, acc, recall, precision, f1 = model.evaluate(X_test, y_test, batch_size=64)

idx = len(results) + 1
results.loc[idx] = ['CNN - Transfer Learning', acc, precision, recall, f1]

print('Test score:', score)

results

 64/369 [====>.........................] - ETA: 0s

128/369 [=========>....................] - ETA: 0s

192/369 [==============>...............] - ETA: 0s

256/369 [===================>..........] - ETA: 0s

320/369 [=========================>....] - ETA: 0s

369/369 [==============================] - 0s     

Test score: 0.236430634732


,Model,Accuracy,Precision,Recall,F1
1,CNN - Transfer Learning,0.943089,0.957622,0.921409,0.939098


In [10]:
model.save('models/Model.h5')